In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import pickle

import seaborn as sns

%matplotlib qt
mpl.rcParams['lines.linewidth'] = 0.91
plt.style.use('seaborn-v0_8-whitegrid')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### Functions to detect bursts in acceleration signal ####

def hl_envelopes_idx(s, dmin=1, dmax=1, split=False):
    """
    Compute high and low envelopes of a signal s
    Parameters
    ----------
    s: 1d-array, data signal from which to extract high and low envelopes
    dmin, dmax: int, optional, size of chunks, use this if the size of the input signal is too big
    split: bool, optional, if True, split the signal in half along its mean, might help to generate the envelope in some cases

    Returns
    -------
    lmin,lmax : high/low envelope idx of input signal s
    """

    # locals min      
    lmin = (np.diff(np.sign(np.diff(s))) > 0).nonzero()[0] + 1 
    # locals max
    lmax = (np.diff(np.sign(np.diff(s))) < 0).nonzero()[0] + 1 
    
    if split:
        # s_mid is zero if s centered around x-axis or more generally mean of signal
        s_mid = np.mean(s) 
        # pre-sorting of locals min based on relative position with respect to s_mid 
        lmin = lmin[s[lmin]<s_mid]
        # pre-sorting of local max based on relative position with respect to s_mid 
        lmax = lmax[s[lmax]>s_mid]

    # global min of dmin-chunks of locals min 
    lmin = lmin[[i+np.argmin(s[lmin[i:i+dmin]]) for i in range(0,len(lmin),dmin)]]
    # global max of dmax-chunks of locals max 
    lmax = lmax[[i+np.argmax(s[lmax[i:i+dmax]]) for i in range(0,len(lmax),dmax)]]
    
    return lmin,lmax

def detect_bursts(acc, envelope = True, plot = False, alfa = 15):
    """
    Detect bursts in acceleration signal

    Parameters
    ----------
    std_acc : pd.Series
        Standard deviation of acceleration signal with a 1 s resolution
    envelope : bool, optional
        If True, detect bursts based on the envelope of the signal
        If False, detect bursts based on the std of the signal

    Returns
    -------
    bursts : pd.Series
        pd.DataFrame with burst start times, end times, and duration
    """

    if envelope:
        lmin, lmax = hl_envelopes_idx(acc.values, dmin=9, dmax=9)
        # adjust shapes
        if len(lmin) > len(lmax):
            lmin = lmin[:-1]
        if len(lmax) > len(lmin):
            lmax = lmax[1:]
        th = np.percentile(acc.values[lmax] - acc.values[lmin], 10) * alfa
        std_acc = pd.Series(acc.values[lmax] - acc.values[lmin], index = acc.index[lmax])
    else:
        std_acc = acc.resample("1 s").std()
        std_acc.index.round("1 s")
        th = np.percentile(std_acc, 10) * alfa

    if plot:
        plt.figure()
        plt.plot(std_acc, color = 'b')
        plt.axhline(th, color = 'r')

    bursts1 = (std_acc > th).astype(int)
    start_burst = bursts1.where(bursts1.diff()==1).dropna()
    end_burst = bursts1.where(bursts1.diff()==-1).dropna()
    if bursts1.iloc[0] == 1:
            start_burst = pd.concat([pd.Series(0, index = [bursts1.index[0]]), start_burst])
    if bursts1.iloc[-1] == 1:
        end_burst = pd.concat([end_burst, pd.Series(0, index = [bursts1.index[-1]])])
    bursts_df = pd.DataFrame({"duration": end_burst.index - start_burst.index}, index = start_burst.index)

    start = bursts_df.index
    end = pd.to_datetime((bursts_df.index + bursts_df["duration"]).values)

    end = end.to_series().reset_index(drop = True)
    start = start.to_series().reset_index(drop = True)

    duration_between_bursts = (start.iloc[1:].values - end.iloc[:-1].values)

    for i in range(len(start)-1):
        if duration_between_bursts[i] < pd.Timedelta("5 s"):
            end[i] = np.nan
            start[i+1] = np.nan
    end.dropna(inplace = True)
    start.dropna(inplace = True)

    # extract amplitude of the bursts
    bursts = pd.DataFrame({"start": start.reset_index(drop = True), "end": end.reset_index(drop = True)})
    burst_amplitude1 = []
    burst_amplitude2 = []
    for i in range(len(bursts)):
        # peak-to-peak amplitude of bp acceleration
        burst_amplitude1.append(acc.loc[bursts["start"].iloc[i]:bursts["end"].iloc[i]].max() - acc.loc[bursts["start"].iloc[i]:bursts["end"].iloc[i]].min())
        # AUC of std_acc
        burst_amplitude2.append(np.trapz(std_acc.loc[bursts["start"].iloc[i]:bursts["end"].iloc[i]]))
    bursts["duration"] = bursts["end"] - bursts["start"]
    bursts["peak-to-peak"] = burst_amplitude1
    bursts["AUC"] = burst_amplitude2
    return bursts

#### Functions to filter bursts that are too close to each other ####

def filter_bursts(data):
    """
    Filter bursts that are neither preceded nor followed by another movement for at least 30 seconds.

    Parameters:
    - data (pd.DataFrame): DataFrame containing 'start', 'end', and 'duration' columns.

    Returns:
    - pd.DataFrame: Filtered DataFrame.
    """
    
    # Calculate the time difference between movements
    data['next_start_diff'] = data['start'].shift(-1) - data['end']
    data['prev_end_diff'] = data['start'] - data['end'].shift(1)
    
    # Convert differences to total seconds for comparison
    data['next_start_diff_seconds'] = data['next_start_diff'].dt.total_seconds()
    data['prev_end_diff_seconds'] = data['prev_end_diff'].dt.total_seconds()
    
    # Filter movements with at least 30 seconds separation from both previous and next movements
    filtered_data = data[(data['next_start_diff_seconds'] > 30) & (data['prev_end_diff_seconds'] > 30)]

    data.drop(columns=['next_start_diff', 'prev_end_diff', 'next_start_diff_seconds', 'prev_end_diff_seconds'], inplace=True)
    
    # Return the filtered data, dropping the temporary columns used for filtering
    return filtered_data.drop(columns=['next_start_diff', 'prev_end_diff', 'next_start_diff_seconds', 'prev_end_diff_seconds'])

#### Functions to find combination of bursts happening at different limbs ####

# For now, implemented for 
# - all 5 limbs together
# - every combination?


def is_isolated(start, end, df):
    # Check if the start or end of an interval falls within any interval in the dataframe
    overlap = df[(df['start'] <= end) & (df['end'] >= start)]
    return overlap.empty

def merge_excluding(current_df):
    df_list = [bursts_ll, bursts_rl, bursts_lw, bursts_rw, bursts_trunk]  # TODO: make this a function argument...
    combined_df = pd.concat([df for df in df_list if not df.equals(current_df)], ignore_index=True)
    return combined_df

def find_isolated_combination(dfs_to_combine, dfs_to_isolate):
    # Merge dataframes that should be combined
    combined_df = pd.concat(dfs_to_combine, ignore_index=True).sort_values(by='start')
    # Merge dataframes from which isolation is required
    isolate_df = pd.concat(dfs_to_isolate, ignore_index=True).sort_values(by='start')

    # Finding overlaps within combined_df
    overlaps = []
    for i, row in combined_df.iterrows():
        overlapping_rows = combined_df[
            (combined_df['start'] <= row['end']) &
            (combined_df['end'] >= row['start']) &
            (combined_df.index != i)
        ]
        if not overlapping_rows.empty:
            # Check isolation from other dataframes
            if is_isolated(row['start'], row['end'], isolate_df):
                overlaps.append(row)

    return pd.DataFrame(overlaps)

def find_combined_movements_all_limbs(dfs):
    # Merging all limb dataframes
    merged_df = pd.concat(dfs, ignore_index=True)
    # Sorting by start time
    merged_df.sort_values(by='start', inplace=True)
    
    # Finding overlapping intervals for all limbs
    overlaps = []
    current_overlap = None
    for index, row in merged_df.iterrows():
        if current_overlap is None:
            current_overlap = {
                'start': row['start'],
                'end': row['end'],
                'limbs_involved': {row['limb']}
            }
        else:
            # Check if the current row overlaps with the current overlapping period
            if row['start'] <= current_overlap['end']:
                current_overlap['limbs_involved'].add(row['limb'])
                # Update the end time to the latest end time
                if row['end'] > current_overlap['end']:
                    current_overlap['end'] = row['end']
            else:
                # Check if the previous overlap involved all limbs
                if current_overlap['limbs_involved'] == {'lw', 'rw', 'll', 'rl', 'trunk'}:
                    overlaps.append(current_overlap)
                # Start a new overlap
                current_overlap = {
                    'start': row['start'],
                    'end': row['end'],
                    'limbs_involved': {row['limb']}
                }
    
    # Final check at the end of the loop
    if current_overlap and current_overlap['limbs_involved'] == {'lw', 'rw', 'll', 'rl', 'trunk'}:
        overlaps.append(current_overlap)
    
    return pd.DataFrame(overlaps)

In [ ]:
# Description: This script detects bursts in the accelerometer data of the trunk and limbs of the subjects, and save the results in a pickle file. 
# The bursts are detected using the Hilbert envelope method, and the isolated movements are extracted for each limb. 
# The script also computes the area under the curve of the Hilbert envelope for each burst, and detects posture changes from the trunk accelerometer data. 
# The results are saved in a dictionary with keys for each combination of limbs.

import numpy as np
import pandas as pd
import neurokit2 as nk
import pickle

from functions.acc_utils import compute_acc_norm
from functions.posture import compute_spherical_coordinates, detect_posture_changes


diary_SPT = {    
    "158": [pd.Timestamp('2024-02-28 23:00:00'), pd.Timestamp('2024-02-29 07:15:00')], # 158 OK
    "633": [pd.Timestamp('2024-03-07 00:05:00'), pd.Timestamp('2024-03-07 06:36:00')], # 633 OK
    "906": [pd.Timestamp('2024-03-07 00:30:00'), pd.Timestamp('2024-03-07 07:30:00')], # 906 OK
    "958": [pd.Timestamp('2024-03-13 22:00:00'), pd.Timestamp('2024-03-14 06:00:00')], # 958 OK
    "127": [pd.Timestamp('2024-03-13 23:15:00'), pd.Timestamp('2024-03-14 06:50:00')], # 127 OK
    "098": [pd.Timestamp('2024-03-16 02:01:00'), pd.Timestamp('2024-03-16 09:50:00')], # 098 OK
    "547": [pd.Timestamp('2024-03-16 01:04:00'), pd.Timestamp('2024-03-16 07:40:00')], # 547 OK
    "815": [pd.Timestamp('2024-03-20 23:00:00'), pd.Timestamp('2024-03-21 07:30:00')], # 815 OK
    "914": [pd.Timestamp('2024-03-20 21:50:00'), pd.Timestamp('2024-03-21 05:50:00')], # 914 OK
    "971": [pd.Timestamp('2024-03-20 23:50:00'), pd.Timestamp('2024-03-21 07:50:00')], # 971 OK
    "279": [pd.Timestamp('2024-03-28 00:10:00'), pd.Timestamp('2024-03-28 07:27:00')], # 279 OK
    "965": [pd.Timestamp('2024-03-28 01:25:00'), pd.Timestamp('2024-03-28 09:20:00')], # 965 OK
}

diary_TIB = {
    "158": [pd.Timestamp('2024-02-28 22:15:00'), pd.Timestamp('2024-02-29 07:45:00')], # 158 OK
    "633": [pd.Timestamp('2024-03-06 23:39:00'), pd.Timestamp('2024-03-07 08:00:00')], # 633 OK 
    "906": [pd.Timestamp('2024-03-07 00:15:00'), pd.Timestamp('2024-03-07 07:35:00')], # 906 OK
    "958": [pd.Timestamp('2024-03-13 21:30:00'), pd.Timestamp('2024-03-14 06:30:00')], # 958 OK
    "127": [pd.Timestamp('2024-03-13 22:00:00'), pd.Timestamp('2024-03-14 07:10:00')], # 127 OK 
    "098": [pd.Timestamp('2024-03-16 01:49:00'), pd.Timestamp('2024-03-16 09:52:00')], # 098 OK 
    "547": [pd.Timestamp('2024-03-16 00:26:00'), pd.Timestamp('2024-03-16 08:20:00')], # 547 OK 
    "815": [pd.Timestamp('2024-03-20 22:00:00'), pd.Timestamp('2024-03-21 07:30:00')], # 815 OK 
    "914": [pd.Timestamp('2024-03-20 21:30:00'), pd.Timestamp('2024-03-21 06:20:00')], # 914 OK 
    "971": [pd.Timestamp('2024-03-20 23:30:00'), pd.Timestamp('2024-03-21 08:08:00')], # 971 OK 
    "279": [pd.Timestamp('2024-03-28 00:04:00'), pd.Timestamp('2024-03-28 07:41:00')], # 279 OK
    "965": [pd.Timestamp('2024-03-28 01:22:00'), pd.Timestamp('2024-03-28 09:22:00')], # 965 OK
}

# subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815"]
subjects = ["127", "914", "965"]

for i, sub in enumerate(subjects):

    print(sub)

    with open(f'/Volumes/Untitled/rehab/data/{sub}/ax_data.pkl', 'rb') as f:
        ax_data = pickle.load(f)

    print("Loaded ax_data!")

    trunk_df = pd.Series(compute_acc_norm(ax_data["trunk"][["x", "y", "z"]].values), index = pd.to_datetime(ax_data["trunk"]["time"], unit = "s") + pd.Timedelta(hours = 1))
    ll_df = pd.Series(compute_acc_norm(ax_data["la"][["x", "y", "z"]].values), index = pd.to_datetime(ax_data["la"]["time"], unit = "s") + pd.Timedelta(hours = 1))
    rl_df = pd.Series(compute_acc_norm(ax_data["ra"][["x", "y", "z"]].values), index = pd.to_datetime(ax_data["ra"]["time"], unit = "s") + pd.Timedelta(hours = 1))
    lw_df = pd.Series(compute_acc_norm(ax_data["lw"][["x", "y", "z"]].values), index = pd.to_datetime(ax_data["lw"]["time"], unit = "s") + pd.Timedelta(hours = 1))
    rw_df = pd.Series(compute_acc_norm(ax_data["rw"][["x", "y", "z"]].values), index = pd.to_datetime(ax_data["rw"]["time"], unit = "s") + pd.Timedelta(hours = 1))

    start_sleep, end_sleep = diary_TIB[sub]

    trunk_df = trunk_df.loc[start_sleep:end_sleep]
    ll_df = ll_df.loc[start_sleep:end_sleep]
    rl_df = rl_df.loc[start_sleep:end_sleep]
    lw_df = lw_df.loc[start_sleep:end_sleep]
    rw_df = rw_df.loc[start_sleep:end_sleep]

    # TODO: Modify sampling rate to 100 Hz

    lw_df_bp = pd.Series(nk.signal_filter(lw_df.values, sampling_rate = 50, lowcut=0.1, highcut=5, method='butterworth', order=8), index = lw_df.index)
    rw_df_bp = pd.Series(nk.signal_filter(rw_df.values, sampling_rate = 50, lowcut=0.1, highcut=5, method='butterworth', order=8), index = rw_df.index)
    ll_df_bp = pd.Series(nk.signal_filter(ll_df.values, sampling_rate = 50, lowcut=0.1, highcut=5, method='butterworth', order=8), index = ll_df.index)
    rl_df_bp = pd.Series(nk.signal_filter(rl_df.values, sampling_rate = 50, lowcut=0.1, highcut=5, method='butterworth', order=8), index = rl_df.index)
    trunk_df_bp = pd.Series(nk.signal_filter(trunk_df.values, sampling_rate = 50, lowcut=0.1, highcut=5, method='butterworth', order=8), index = trunk_df.index)
    bursts_lw = detect_bursts(lw_df_bp, plot = False, alfa = 7)
    bursts_rw = detect_bursts(rw_df_bp, plot = False, alfa = 7)
    bursts_ll = detect_bursts(ll_df_bp, plot = False, alfa = 6)
    bursts_rl = detect_bursts(rl_df_bp, plot = False, alfa = 6)
    bursts_trunk = detect_bursts(trunk_df_bp, plot = False, alfa = 5)

    # Isolation checks
    bursts_ll['isolated'] = bursts_ll.apply(lambda x: is_isolated(x['start'], x['end'], merge_excluding(bursts_ll)), axis=1)
    bursts_rl['isolated'] = bursts_rl.apply(lambda x: is_isolated(x['start'], x['end'], merge_excluding(bursts_rl)), axis=1)
    bursts_lw['isolated'] = bursts_lw.apply(lambda x: is_isolated(x['start'], x['end'], merge_excluding(bursts_lw)), axis=1)
    bursts_rw['isolated'] = bursts_rw.apply(lambda x: is_isolated(x['start'], x['end'], merge_excluding(bursts_rw)), axis=1)
    bursts_trunk['isolated'] = bursts_trunk.apply(lambda x: is_isolated(x['start'], x['end'], merge_excluding(bursts_trunk)), axis=1)

    # Extract isolated movements for each limb
    bursts_ll_isolated = bursts_ll[bursts_ll['isolated']]
    bursts_rl_isolated = bursts_rl[bursts_rl['isolated']]
    bursts_lw_isolated = bursts_lw[bursts_lw['isolated']]
    bursts_rw_isolated = bursts_rw[bursts_rw['isolated']]
    bursts_trunk_isolated = bursts_trunk[bursts_trunk['isolated']]

    bursts_wrists_isolated = pd.concat([bursts_lw_isolated, bursts_rw_isolated], ignore_index=True)
    bursts_legs_isolated = pd.concat([bursts_ll_isolated, bursts_rl_isolated], ignore_index=True)

    bursts_both_wrists = find_isolated_combination([bursts_lw, bursts_rw], [bursts_ll, bursts_rl, bursts_trunk]).iloc[::2].reset_index(drop=True)

    # Finding isolated movements for both legs alone (no wrists or trunk)
    bursts_both_legs = find_isolated_combination([bursts_ll, bursts_rl], [bursts_lw, bursts_rw, bursts_trunk]).iloc[::2].reset_index(drop=True)

    bursts_lw["limb"] = "lw"
    bursts_rw["limb"] = "rw"
    bursts_ll["limb"] = "ll"
    bursts_rl["limb"] = "rl"
    bursts_trunk["limb"] = "trunk"
    bursts_all_limbs_combined = find_combined_movements_all_limbs([bursts_lw, bursts_rw, bursts_ll, bursts_rl, bursts_trunk])

    bursts_all_limbs_combined["AUC"] = np.nan

    lmin, lmax = hl_envelopes_idx(lw_df_bp.values, dmin=9, dmax=9)
    if len(lmin) > len(lmax):
        lmin = lmin[:-1]
    if len(lmax) > len(lmin):
        lmax = lmax[1:]
    env_diff_lw = pd.Series(lw_df_bp.values[lmax] - lw_df_bp.values[lmin], index = lw_df_bp.index[lmax])

    lmin, lmax = hl_envelopes_idx(rw_df_bp.values, dmin=9, dmax=9)
    if len(lmin) > len(lmax):
        lmin = lmin[:-1]
    if len(lmax) > len(lmin):
        lmax = lmax[1:]
    env_diff_rw = pd.Series(rw_df_bp.values[lmax] - rw_df_bp.values[lmin], index = rw_df_bp.index[lmax])

    lmin, lmax = hl_envelopes_idx(ll_df_bp.values, dmin=9, dmax=9)
    if len(lmin) > len(lmax):
        lmin = lmin[:-1]
    if len(lmax) > len(lmin):
        lmax = lmax[1:]
    env_diff_ll = pd.Series(ll_df_bp.values[lmax] - ll_df_bp.values[lmin], index = ll_df_bp.index[lmax])

    lmin, lmax = hl_envelopes_idx(rl_df_bp.values, dmin=9, dmax=9)
    if len(lmin) > len(lmax):
        lmin = lmin[:-1]
    if len(lmax) > len(lmin):
        lmax = lmax[1:]
    env_diff_rl = pd.Series(rl_df_bp.values[lmax] - rl_df_bp.values[lmin], index = rl_df_bp.index[lmax])

    lmin, lmax = hl_envelopes_idx(trunk_df_bp.values, dmin=9, dmax=9)
    if len(lmin) > len(lmax):
        lmin = lmin[:-1]
    if len(lmax) > len(lmin):
        lmax = lmax[1:]
    env_diff_trunk = pd.Series(trunk_df_bp.values[lmax] - trunk_df_bp.values[lmin], index = trunk_df_bp.index[lmax])

    for i, b in enumerate(range(len(bursts_all_limbs_combined))):
        bursts_all_limbs_combined.loc[i, "AUC"] = np.trapz(env_diff_lw.loc[bursts_all_limbs_combined["start"].iloc[i]:bursts_all_limbs_combined["end"].iloc[i]]) 
        + np.trapz(env_diff_rw.loc[bursts_all_limbs_combined["start"].iloc[i]:bursts_all_limbs_combined["end"].iloc[i]]) 
        + np.trapz(env_diff_ll.loc[bursts_all_limbs_combined["start"].iloc[i]:bursts_all_limbs_combined["end"].iloc[i]]) 
        + np.trapz(env_diff_rl.loc[bursts_all_limbs_combined["start"].iloc[i]:bursts_all_limbs_combined["end"].iloc[i]]) 
        + np.trapz(env_diff_trunk.loc[bursts_all_limbs_combined["start"].iloc[i]:bursts_all_limbs_combined["end"].iloc[i]]) 

    bursts_both_wrists["AUC"] = np.nan
    for i, b in enumerate(range(len(bursts_both_wrists))):
        bursts_both_wrists.loc[i, "AUC"] = np.trapz(env_diff_lw.loc[bursts_both_wrists["start"].iloc[i]:bursts_both_wrists["end"].iloc[i]]) 
        + np.trapz(env_diff_rw.loc[bursts_both_wrists["start"].iloc[i]:bursts_both_wrists["end"].iloc[i]])

    bursts_both_legs["AUC"] = np.nan
    for i, b in enumerate(range(len(bursts_both_legs))):
        bursts_both_legs.loc[i, "AUC"] = np.trapz(env_diff_ll.loc[bursts_both_legs["start"].iloc[i]:bursts_both_legs["end"].iloc[i]]) 
        + np.trapz(env_diff_rl.loc[bursts_both_legs["start"].iloc[i]:bursts_both_legs["end"].iloc[i]])

    # Trunk - I need xyz
    ax_data['trunk'].index = pd.to_datetime(ax_data['trunk']['time'], unit='s') + pd.Timedelta(hours = 1)
    ax_data['trunk'].drop(columns=['time'], inplace=True)
    trunk_acc_df = ax_data['trunk'].loc[start_sleep:end_sleep]
    del ax_data

    phi, theta = compute_spherical_coordinates(trunk_acc_df.resample('10s').median())
    trunk_acc_sph = pd.DataFrame({"phi": phi * 180 / np.pi, "theta": theta * 180 / np.pi}, index=trunk_acc_df.resample('10s').median().index)
    updated_df = detect_posture_changes(trunk_acc_sph.copy())
    time_posture_change30 = updated_df[updated_df['posture_change30']].index
    time_posture_change10 = updated_df[updated_df['posture_change10']].index
    # join bursts from all limbs and posture changes

    bursts_all_limbs_combined["posture_change"] = np.nan

    for time in time_posture_change10:
        for i in range(len(bursts_all_limbs_combined)):
                if time > bursts_all_limbs_combined["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_all_limbs_combined["end"].iloc[i]+pd.Timedelta(seconds = 5):
                    bursts_all_limbs_combined["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees10"]
    # join bursts and posture changes

    bursts_lw["posture_change"] = np.nan
    bursts_rw["posture_change"] = np.nan
    bursts_ll["posture_change"] = np.nan
    bursts_rl["posture_change"] = np.nan
    bursts_trunk["posture_change"] = np.nan

    for time in time_posture_change30:
        for i in range(len(bursts_lw)):
            if time > bursts_lw["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_lw["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_lw["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees30"]
        for i in range(len(bursts_rw)):
            if time > bursts_rw["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_rw["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_rw["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees30"]
        for i in range(len(bursts_ll)):
            if time > bursts_ll["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_ll["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_ll["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees30"]
        for i in range(len(bursts_rl)):
            if time > bursts_rl["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_rl["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_rl["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees30"]
        for i in range(len(bursts_trunk)):
            if time > bursts_trunk["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_trunk["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_trunk["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees30"]

    for time in time_posture_change10:
        for i in range(len(bursts_lw)):
            if time > bursts_lw["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_lw["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_lw["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees10"]
        for i in range(len(bursts_rw)):
            if time > bursts_rw["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_rw["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_rw["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees10"]
        for i in range(len(bursts_ll)):
            if time > bursts_ll["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_ll["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_ll["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees10"]
        for i in range(len(bursts_rl)):
            if time > bursts_rl["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_rl["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_rl["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees10"]
        for i in range(len(bursts_trunk)):
            if time > bursts_trunk["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_trunk["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_trunk["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees10"]

    # summarize all the bursts in a dict, with a key for each combination of limbs

    bursts = {
        "lw": bursts_lw,
        "rw": bursts_rw,
        "ll": bursts_ll,
        "rl": bursts_rl,
        "trunk": bursts_trunk,
        "wrists": bursts_wrists_isolated,
        "legs": bursts_legs_isolated,
        "trunk_isolated": bursts_trunk_isolated,
        "both_wrists": bursts_both_wrists,
        "both_legs": bursts_both_legs,
        "all_limbs": bursts_all_limbs_combined
    }

    # SAVE
    with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_TIB.pkl', 'wb') as f:
        pickle.dump(bursts, f)

In [3]:
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815"]

for i, sub in enumerate(["815"]):
    with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_TIB.pkl', 'rb') as f:
        bursts = pickle.load(f)

    bursts_lw = bursts["lw"]
    bursts_rw = bursts["rw"]
    bursts_ll = bursts["ll"]
    bursts_rl = bursts["rl"]
    bursts_trunk = bursts["trunk"]
    break

In [4]:
plt.figure(figsize=(19, 12))
plt.subplot(5, 1, 1)
# plt.plot(lw_df)
for i in range(len(bursts_lw)):
    plt.axvspan(bursts_lw["start"].iloc[i], bursts_lw["end"].iloc[i], color = 'b', alpha = 0.3)
plt.ylabel("ACC (g)", fontsize = 16)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.legend(["LW ACC", "Movement"], loc = "upper right", fontsize = 16)

plt.subplot(5, 1, 2, sharex = plt.subplot(5, 1, 1), sharey = plt.subplot(5, 1, 1))
# plt.plot(rw_df)
for i in range(len(bursts_rw)):
    plt.axvspan(bursts_rw["start"].iloc[i], bursts_rw["end"].iloc[i], color = 'b', alpha = 0.3)
plt.ylabel("ACC (g)", fontsize = 16)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.legend(["RW ACC", "Movement"], loc = "upper right", fontsize = 16)

plt.subplot(5, 1, 3, sharex = plt.subplot(5, 1, 1), sharey = plt.subplot(5, 1, 1))
# plt.plot(ll_df)
for i in range(len(bursts_ll)):
    plt.axvspan(bursts_ll["start"].iloc[i], bursts_ll["end"].iloc[i], color = 'b', alpha = 0.3)
plt.ylabel("ACC (g)", fontsize = 16)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.legend(["LL ACC", "Movement"], loc = "upper right", fontsize = 16)

plt.subplot(5, 1, 4, sharex = plt.subplot(5, 1, 1), sharey = plt.subplot(5, 1, 1))
# plt.plot(rl_df)
for i in range(len(bursts_rl)):
    plt.axvspan(bursts_rl["start"].iloc[i], bursts_rl["end"].iloc[i], color = 'b', alpha = 0.3)
plt.ylabel("ACC (g)", fontsize = 16)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.legend(["RL ACC", "Movement"], loc = "upper right", fontsize = 16)

plt.subplot(5, 1, 5, sharex = plt.subplot(5, 1, 1), sharey = plt.subplot(5, 1, 1))
# plt.plot(trunk_df)
for i in range(len(bursts_trunk)):
    plt.axvspan(bursts_trunk["start"].iloc[i], bursts_trunk["end"].iloc[i], color = 'b', alpha = 0.3)
plt.ylabel("ACC (g)", fontsize = 16)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.legend(["Trunk ACC", "Movement"], loc = "upper right", fontsize = 16)

2024-06-30 23:22:04.908 python[1030:9686] IMKClient Stall detected, *please Report* your user scenario attaching a spindump (or sysdiagnose) that captures the problem - (imkxpc_bundleIdentifierWithReply:) block performed very slowly (3.52 secs).


## All subjects

In [4]:
diary_SPT = {    
    "158": [pd.Timestamp('2024-02-28 23:00:00'), pd.Timestamp('2024-02-29 07:15:00')], # 158 OK
    "633": [pd.Timestamp('2024-03-07 00:05:00'), pd.Timestamp('2024-03-07 06:36:00')], # 633 OK
    "906": [pd.Timestamp('2024-03-07 00:30:00'), pd.Timestamp('2024-03-07 07:30:00')], # 906 OK
    "958": [pd.Timestamp('2024-03-13 22:00:00'), pd.Timestamp('2024-03-14 06:00:00')], # 958 OK
    "127": [pd.Timestamp('2024-03-13 23:15:00'), pd.Timestamp('2024-03-14 06:50:00')], # 127 OK
    "098": [pd.Timestamp('2024-03-16 02:01:00'), pd.Timestamp('2024-03-16 09:50:00')], # 098 OK
    "547": [pd.Timestamp('2024-03-16 01:04:00'), pd.Timestamp('2024-03-16 07:40:00')], # 547 OK
    "815": [pd.Timestamp('2024-03-20 23:00:00'), pd.Timestamp('2024-03-21 07:30:00')], # 815 OK
    "914": [pd.Timestamp('2024-03-20 21:50:00'), pd.Timestamp('2024-03-21 05:50:00')], # 914 OK
    "971": [pd.Timestamp('2024-03-20 23:50:00'), pd.Timestamp('2024-03-21 07:50:00')], # 971 OK
    "279": [pd.Timestamp('2024-03-28 00:10:00'), pd.Timestamp('2024-03-28 07:27:00')], # 279 OK
    "965": [pd.Timestamp('2024-03-28 01:25:00'), pd.Timestamp('2024-03-28 09:20:00')], # 965 OK
}

diary_TIB = {
    "158": [pd.Timestamp('2024-02-28 22:15:00'), pd.Timestamp('2024-02-29 07:45:00')], # 158 OK
    "633": [pd.Timestamp('2024-03-06 23:39:00'), pd.Timestamp('2024-03-07 08:00:00')], # 633 OK 
    "906": [pd.Timestamp('2024-03-07 00:15:00'), pd.Timestamp('2024-03-07 07:35:00')], # 906 OK
    "958": [pd.Timestamp('2024-03-13 21:30:00'), pd.Timestamp('2024-03-14 06:30:00')], # 958 OK
    "127": [pd.Timestamp('2024-03-13 22:00:00'), pd.Timestamp('2024-03-14 07:10:00')], # 127 OK 
    "098": [pd.Timestamp('2024-03-16 01:49:00'), pd.Timestamp('2024-03-16 09:52:00')], # 098 OK 
    "547": [pd.Timestamp('2024-03-16 00:26:00'), pd.Timestamp('2024-03-16 08:20:00')], # 547 OK 
    "815": [pd.Timestamp('2024-03-20 22:00:00'), pd.Timestamp('2024-03-21 07:30:00')], # 815 OK 
    "914": [pd.Timestamp('2024-03-20 21:30:00'), pd.Timestamp('2024-03-21 06:20:00')], # 914 OK 
    "971": [pd.Timestamp('2024-03-20 23:30:00'), pd.Timestamp('2024-03-21 08:08:00')], # 971 OK 
    "279": [pd.Timestamp('2024-03-28 00:04:00'), pd.Timestamp('2024-03-28 07:41:00')], # 279 OK
    "965": [pd.Timestamp('2024-03-28 01:22:00'), pd.Timestamp('2024-03-28 09:22:00')], # 965 OK
}

In [5]:
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815", "127", "914", "965"]

In [6]:
from collections import Counter

import pyreadr

from functions.bursts import characterize_bursts

In [67]:
part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms3.out/"
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815"]

SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

bursts_lw = {sub: 0 for sub in subjects}
bursts_rw = {sub: 0 for sub in subjects}
bursts_ll = {sub: 0 for sub in subjects}
bursts_rl = {sub: 0 for sub in subjects}
bursts_trunk = {sub: 0 for sub in subjects}
bursts_all_limbs = {sub: 0 for sub in subjects}
bursts_all_limbs_new = {sub: 0 for sub in subjects}
SIB = {sub: 0 for sub in subjects}

limbs_combinations_tot = {sub: 0 for sub in subjects}
limbs_combinations_sleep = {sub: 0 for sub in subjects}
limbs_combinations_awake = {sub: 0 for sub in subjects}

main_movements = [{'LL', 'LW', 'RL', 'RW', 'T'},
{'LW'},
{'RW'},
{'RL'},
{'LL'},
{'LL', 'RL', 'T'}]

bursts_df = pd.DataFrame()

for i, sub in enumerate(subjects):
    SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
    # end time - onset time
    SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]
    # onset time - previous end time
    # print(min(SIB_all[sub].dropna()["awake.duration"]))
    # SIB[sub]['sib.onset.time'] += pd.Timedelta('5s')
    # SIB[sub]['sib.end.time'] += pd.Timedelta('5s')
    with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_TIB.pkl', 'rb') as f:
        bursts = pickle.load(f)
    # bursts_lw[sub] = bursts["lw"]
    # bursts_rw[sub] = bursts["rw"]
    # bursts_ll[sub] = bursts["ll"]
    # bursts_rl[sub] = bursts["rl"]
    # bursts_trunk[sub] = bursts["trunk"]
    # bursts_all_limbs[sub] = bursts["all_limbs"]

    df_merged_intervals = characterize_bursts(bursts)
    df_merged_intervals["Limbs"] = df_merged_intervals["Limbs"].apply(lambda x: x if set(x) in main_movements else "X")
    spt_start = diary_SPT[sub][0] - pd.Timedelta('5 min')
    spt_end = diary_SPT[sub][1] + pd.Timedelta('5 min')

    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)

    # Take df_merged_intervals between spt_start and spt_end
    df_merged_intervals = df_merged_intervals[(df_merged_intervals["Start"] >= spt_start) & (df_merged_intervals["End"] <= spt_end)].reset_index(drop=True) 

    SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]
    SIB[sub]["sub_ID"] = sub

    df_merged_intervals["SIB"] = 0
    for i, row in SIB[sub].iterrows():
        df_merged_intervals.loc[(df_merged_intervals["Start"] >= row["sib.onset.time"] + pd.Timedelta("5s")) & (df_merged_intervals["End"] <= row["sib.end.time"] - pd.Timedelta("5s")), "SIB"] = 1

    df_merged_intervals["sub_ID"] = sub

    # start_sleep = diary_SPT[sub][0] - pd.Timedelta('10 min')
    # end_sleep = diary_SPT[sub][1] + pd.Timedelta('10 min')

    # df_merged_intervals = df_merged_intervals.loc[(df_merged_intervals["Start"] >= start_sleep) & (df_merged_intervals["End"] <= end_sleep)]

    bursts_df = pd.concat([bursts_df, df_merged_intervals])

    limbs_comb_tot = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals["Limbs"])
    limbs_combinations_tot_df = pd.DataFrame(limbs_comb_tot.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_tot_df["sub_ID"] = sub
    limbs_combinations_tot[sub] = limbs_combinations_tot_df

    # Sleep
    limbs_combination_sleep = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals[df_merged_intervals["SIB"] == 1]["Limbs"])
    limbs_combination_sleep_df = pd.DataFrame(limbs_combination_sleep.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combination_sleep_df["sub_ID"] = sub
    limbs_combinations_sleep[sub] = limbs_combination_sleep_df

    # Awake
    limbs_combination_awake = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals[df_merged_intervals["SIB"] == 0]["Limbs"])
    limbs_combination_awake_df = pd.DataFrame(limbs_combination_awake.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combination_awake_df["sub_ID"] = sub
    limbs_combinations_awake[sub] = limbs_combination_awake_df

bursts_df = bursts_df.reset_index(drop=True)

In [68]:
print(bursts_df.shape[0])

print(bursts_df[bursts_df["SIB"] == 1].shape[0])

print(bursts_df[bursts_df["SIB"] == 0].shape[0])

1888
1193
695


In [69]:
n_tot_bursts_sleep = bursts_df[bursts_df["SIB"] == 1].shape[0]
n_tot_bursts_awake = bursts_df[bursts_df["SIB"] == 0].shape[0]

In [70]:
limbs_combinations_tot_df_ALL = pd.concat(limbs_combinations_tot .values(), ignore_index=True)
limbs_combinations_sleep_df_ALL = pd.concat(limbs_combinations_sleep .values(), ignore_index=True)
limbs_combinations_awake_df_ALL = pd.concat(limbs_combinations_awake .values(), ignore_index=True)
print(limbs_combinations_tot_df_ALL["Count"].sum())
print(limbs_combinations_sleep_df_ALL["Count"].sum())
print(limbs_combinations_awake_df_ALL["Count"].sum())

1888
1193
695


In [76]:
bursts_lw = {sub: 0 for sub in subjects}
bursts_rw = {sub: 0 for sub in subjects}
bursts_ll = {sub: 0 for sub in subjects}
bursts_rl = {sub: 0 for sub in subjects}
bursts_trunk = {sub: 0 for sub in subjects}
SIB = {sub: 0 for sub in subjects}

limbs_combinations_tot = {sub: 0 for sub in subjects}
limbs_combinations_sleep = {sub: 0 for sub in subjects}
limbs_combinations_awake = {sub: 0 for sub in subjects}
limbs_combinations1part_sleep = {sub: 0 for sub in subjects}
limbs_combinations1part_awake = {sub: 0 for sub in subjects}
limbs_combinations2part_sleep = {sub: 0 for sub in subjects}
limbs_combinations2part_awake = {sub: 0 for sub in subjects}

part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms3.out/"
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815",]# "127", "914", "965"]

SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

main_movements = [{'LL', 'LW', 'RL', 'RW', 'T'},
{'LW'},
{'RW'},
{'RL'},
{'LL'},
{'LL', 'RL', 'T'}]

for i, sub in enumerate(subjects):
    if sub in ["127", "914", "965"]:
        with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_TIB.pkl', 'rb') as f:
            bursts = pickle.load(f)
    else:
        with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts.pkl', 'rb') as f:
            bursts = pickle.load(f)

    # df_merged:intervals containts all the bursts 
    df_merged_intervals = characterize_bursts(bursts)

    # Optional: replace any other limb combinations with 'Other'
    df_merged_intervals["Limbs"] = df_merged_intervals["Limbs"].apply(lambda x: x if set(x) in main_movements else "X")

    # Add SIB information from GGIR
    SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]

    spt_start = diary_SPT[sub][0] - pd.Timedelta('5 min')
    spt_end = diary_SPT[sub][1] + pd.Timedelta('5 min')

    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    
    SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]

    SIB[sub]["sub_ID"] = sub

    # Find bursts that overlap with SIB
    df_merged_intervals["SIB"] = 0
    for i, row in SIB[sub].iterrows():
        df_merged_intervals.loc[(df_merged_intervals["Start"] >= row["sib.onset.time"] + pd.Timedelta("5s")) & (df_merged_intervals["End"] <= row["sib.end.time"] - pd.Timedelta("5s")), "SIB"] = 1

    #### Count the number of occurrences of each limb combination ####

    # Total
    limbs_comb_tot = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals["Limbs"])
    limbs_combinations_tot_df = pd.DataFrame(limbs_comb_tot.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_tot_df["sub_ID"] = sub
    limbs_combinations_tot[sub] = limbs_combinations_tot_df

    # Sleep
    limbs_combination_sleep = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals[df_merged_intervals["SIB"] == 1]["Limbs"])
    limbs_combination_sleep_df = pd.DataFrame(limbs_combination_sleep.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combination_sleep_df["sub_ID"] = sub
    limbs_combinations_sleep[sub] = limbs_combination_sleep_df

    # Awake
    limbs_combination_awake = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals[df_merged_intervals["SIB"] == 0]["Limbs"])
    limbs_combination_awake_df = pd.DataFrame(limbs_combination_awake.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combination_awake_df["sub_ID"] = sub
    limbs_combinations_awake[sub] = limbs_combination_awake_df

    #### First part VS second part of the night ####
    start_sleep, end_sleep = diary_SPT[sub]
    sleep_midpoint = start_sleep + (end_sleep - start_sleep) / 2

    df_merged_intervals_1 = df_merged_intervals[df_merged_intervals["Start"] < sleep_midpoint]
    df_merged_intervals_2 = df_merged_intervals[df_merged_intervals["Start"] >= sleep_midpoint]

    limbs_comb_1_sleep = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_1[df_merged_intervals_1["SIB"] == 1]["Limbs"])
    limbs_comb_1_awake = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_1[df_merged_intervals_1["SIB"] == 0]["Limbs"])
    limbs_comb_2_sleep = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_2[df_merged_intervals_2["SIB"] == 1]["Limbs"])
    limbs_comb_2_awake = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_2[df_merged_intervals_2["SIB"] == 0]["Limbs"])

    limbs_combinations_df_1_sleep = pd.DataFrame(limbs_comb_1_sleep.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_1_sleep["sub_ID"] = sub
    limbs_combinations_df_1_awake = pd.DataFrame(limbs_comb_1_awake.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_1_awake["sub_ID"] = sub

    limbs_combinations_df_2_sleep = pd.DataFrame(limbs_comb_2_sleep.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_2_sleep["sub_ID"] = sub
    limbs_combinations_df_2_awake = pd.DataFrame(limbs_comb_2_awake.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_2_awake["sub_ID"] = sub

    limbs_combinations1part_sleep[sub] = limbs_combinations_df_1_sleep
    limbs_combinations1part_awake[sub] = limbs_combinations_df_1_awake
    limbs_combinations2part_sleep[sub] = limbs_combinations_df_2_sleep
    limbs_combinations2part_awake[sub] = limbs_combinations_df_2_awake

In [77]:
limbs_combinations_tot_df_ALL = pd.concat(limbs_combinations_tot.values(), ignore_index=True)
limbs_combinations_sleep_df_ALL = pd.concat(limbs_combinations_sleep.values(), ignore_index=True)
limbs_combinations_awake_df_ALL = pd.concat(limbs_combinations_awake.values(), ignore_index=True)
limbs_combinations1part_sleep_df_ALL = pd.concat(limbs_combinations1part_sleep .values(), ignore_index=True)
limbs_combinations1part_awake_df_ALL = pd.concat(limbs_combinations1part_awake .values(), ignore_index=True)
limbs_combinations2part_sleep_df_ALL = pd.concat(limbs_combinations2part_sleep .values(), ignore_index=True)
limbs_combinations2part_awake_df_ALL = pd.concat(limbs_combinations2part_awake .values(), ignore_index=True)

Subject-wise

In [78]:
limbs_comb_groupby = limbs_combinations_tot_df_ALL.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_sleep_groupby = limbs_combinations_sleep_df_ALL.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_awake_groupby = limbs_combinations_awake_df_ALL.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)

sns.set_context("talk")

# Plot this information
plt.figure(figsize=(12, 6))
sns.barplot(data = limbs_comb_groupby.reset_index(), x = "sub_ID", y = "Count", hue = "Limbs")
plt.title("SPT")

plt.figure(figsize=(12, 6))
sns.barplot(data = limbs_comb_sleep_groupby.reset_index(), x = "sub_ID", y = "Count", hue = "Limbs")
plt.title("Sleep")

plt.figure(figsize=(12, 6))
sns.barplot(data = limbs_comb_awake_groupby.reset_index(), x = "sub_ID", y = "Count", hue = "Limbs")
plt.title("Awake")

Text(0.5, 1.0, 'Awake')

Statistical test (chi squared)

In [79]:
fiveL_sleep = limbs_comb_sleep_groupby["Count"].unstack()[limbs_comb_sleep_groupby["Count"].unstack().index == limbs_comb_sleep_groupby["Count"].unstack().index[1]]
fiveL_wake = limbs_comb_awake_groupby["Count"].unstack()[limbs_comb_awake_groupby["Count"].unstack().index == limbs_comb_awake_groupby["Count"].unstack().index[1]]
noFiveL_sleep = limbs_comb_sleep_groupby["Count"].unstack()[limbs_comb_sleep_groupby["Count"].unstack().index != limbs_comb_sleep_groupby["Count"].unstack().index[1]].sum()
noFiveL_wake = limbs_comb_awake_groupby["Count"].unstack()[limbs_comb_awake_groupby["Count"].unstack().index != limbs_comb_awake_groupby["Count"].unstack().index[1]]

In [80]:
limbs_comb_sleep_groupby.groupby("sub_ID").sum().sum().values, n_tot_bursts_sleep

(array([1159]), 1193)

In [81]:
fiveL_sleep = fiveL_sleep.T / limbs_comb_sleep_groupby.groupby("sub_ID").sum().values
fiveL_wake = fiveL_wake.T / limbs_comb_awake_groupby.groupby("sub_ID").sum().values

# combined l_sleep and l_wake
l_sleep_and_wake = pd.concat([fiveL_sleep.T, fiveL_wake.T], axis = 0, ignore_index=True).T
l_sleep_and_wake.columns = ["Sleep", "Wake"]
l_sleep_and_wake

plt.figure(figsize=(12, 6))
sns.boxplot(data = l_sleep_and_wake, palette = "Set2")
plt.title("Percentage of 5 limbs movements in sleep vs wake")
plt.ylabel("Percentage of 5 limbs movements (%)")

print("Sleep mean and std: ", l_sleep_and_wake["Sleep"].mean()*100, l_sleep_and_wake["Sleep"].std()*100)
print("Wake mean and std: ", l_sleep_and_wake["Wake"].mean()*100, l_sleep_and_wake["Wake"].std()*100)

# Perform wilcoxon signed-rank test
from scipy.stats import wilcoxon

data1 = l_sleep_and_wake["Sleep"]
data2 = l_sleep_and_wake["Wake"]

stat, p = wilcoxon(data1, data2)
print('Statistics=%.3f, p=%.10f' % (stat, p))

Sleep mean and std:  27.79251638626229 11.965187739598703
Wake mean and std:  64.88047571348788 7.76453873060681
Statistics=0.000, p=0.0039062500


Across subjects

In [60]:
l_sleep_and_wake.to_csv("chiSquared_GPT.csv")

In [50]:
plt.figure(figsize = (8,8))
# Whole SPT
plt.pie(limbs_combinations_tot_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count'], 
        labels = limbs_combinations_tot_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False).index, 
        autopct='%1.1f%%', textprops={'fontsize': 19});
plt.title("SPT", fontsize = 21)
plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/pie_wholeSPT.png", dpi = 300, bbox_inches = 'tight')

# Sleep
plt.figure(figsize = (8,8))
plt.pie(limbs_combinations_sleep_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count'], 
        labels = limbs_combinations_sleep_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False).index, 
        autopct='%1.1f%%', textprops={'fontsize': 19});
plt.title("Sleep", fontsize = 21)
plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/pie_Wholesleep.png", dpi = 300, bbox_inches = 'tight')

# Awake
plt.figure(figsize = (8,8))
plt.pie(limbs_combinations_awake_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count'], 
        labels = limbs_combinations_awake_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False).index, 
        autopct='%1.1f%%', textprops={'fontsize': 19});
plt.title("Wake", fontsize = 21)
plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/pie_Wholewake.png", dpi = 300, bbox_inches = 'tight')

In [14]:
# First part of the night sleep
plt.figure(figsize = (8,8))
plt.pie(limbs_combinations1part_sleep_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count'], 
        labels = limbs_combinations1part_sleep_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False).index, 
        autopct='%1.1f%%', textprops={'fontsize': 19});
plt.title("First part of the night - sleep", fontsize = 21)
plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/pie_1half_sleep.png", dpi = 300, bbox_inches = 'tight')

# First part of the night awake
plt.figure(figsize = (8,8))
plt.pie(limbs_combinations1part_awake_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count'], 
        labels = limbs_combinations1part_awake_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False).index, 
        autopct='%1.1f%%', textprops={'fontsize': 19});
plt.title("First part of the night - wake", fontsize = 21)
plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/pie_1half_wake.png", dpi = 300, bbox_inches = 'tight')

# Second part of the night sleep
plt.figure(figsize = (8,8))
plt.pie(limbs_combinations2part_sleep_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count'], 
        labels = limbs_combinations2part_sleep_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False).index, 
        autopct='%1.1f%%', textprops={'fontsize': 19});
plt.title("Second part of the night - sleep", fontsize = 21)
plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/pie_2half_sleep.png", dpi = 300, bbox_inches = 'tight')

# Second part of the night awake
plt.figure(figsize = (8,8))
plt.pie(limbs_combinations2part_awake_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count'], 
        labels = limbs_combinations2part_awake_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False).index, 
        autopct='%1.1f%%', textprops={'fontsize': 19});
plt.title("Second part of the night - wake", fontsize = 21)
plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/pie_2half_wake.png", dpi = 300, bbox_inches = 'tight')


In [18]:
plt.close("all")

2024-06-16 10:26:46.346 python[61372:1077509] IMKClient Stall detected, *please Report* your user scenario attaching a spindump (or sysdiagnose) that captures the problem - (imkxpc_bundleIdentifierWithReply:) block performed very slowly (1.08 secs).


Summarize this info considering only main movements

Total number of possible combinations
- 1 limb (size 1): 5 combinations, one for each element.
- 2 limbs (size 2): 10 combinations, each pair of elements.
- 3 limbs (size 3): 10 combinations, each trio of elements.
- 4 limbs (size 4): 5 combinations, each group excluding one element.
- 5 limbs (size 5): 1 combination, all elements together.

Total of 5+10+10+5+1 = 31 combinations

### Movement bursts distribution throughout the night

In [100]:
bursts_lw = {sub: 0 for sub in subjects}
bursts_rw = {sub: 0 for sub in subjects}
bursts_ll = {sub: 0 for sub in subjects}
bursts_rl = {sub: 0 for sub in subjects}
bursts_trunk = {sub: 0 for sub in subjects}
SIB = {sub: 0 for sub in subjects}
limbs_combinations1part = {sub: 0 for sub in subjects}
limbs_combinations2part = {sub: 0 for sub in subjects}
limbs_combinations3part = {sub: 0 for sub in subjects}
limbs_combinations1part_sleep = {sub: 0 for sub in subjects}
limbs_combinations1part_awake = {sub: 0 for sub in subjects}
limbs_combinations2part_sleep = {sub: 0 for sub in subjects}
limbs_combinations2part_awake = {sub: 0 for sub in subjects}
sleep_time_part1 = {sub: 0 for sub in subjects}
sleep_time_part2 = {sub: 0 for sub in subjects}

part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms3.out/"
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815"]# "127", "914", "965"]

SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

main_movements = [{'LL', 'LW', 'RL', 'RW', 'T'},
{'LW'},
{'RW'},
{'RL'},
{'LL'},
{'LL', 'RL', 'T'}]

for i, sub in enumerate(subjects):
    if sub in ["127", "914", "965"]:
        with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_TIB.pkl', 'rb') as f:
            bursts = pickle.load(f)
    else:
        with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts.pkl', 'rb') as f:
            bursts = pickle.load(f)
    
    df_merged_intervals = characterize_bursts(bursts)

    # Movement bursts distribution throughout the night
    start_sleep, end_sleep = diary_SPT[sub]

    #### DIVIDE SLEEP in two parts ####

    sleep_midpoint = start_sleep + (end_sleep - start_sleep) / 2
    # print(start_sleep, end_sleep, sleep_midpoint)

    df_merged_intervals["Limbs"] = df_merged_intervals["Limbs"].apply(lambda x: x if set(x) in main_movements else "X")

    SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]

    spt_start = diary_SPT[sub][0] - pd.Timedelta('10 min')
    spt_end = diary_SPT[sub][1] + pd.Timedelta('10 min')

    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)

    SIB[sub]["part"] = np.where(SIB[sub]["sib.onset.time"] < sleep_midpoint, "First Half", "Second Half")
    sleep_time_part1[sub] = SIB[sub][SIB[sub]["part"] == "First Half"]["sib.duration"].sum()
    sleep_time_part2[sub] = SIB[sub][SIB[sub]["part"] == "Second Half"]["sib.duration"].sum()
    
    SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]

    SIB[sub]["sub_ID"] = sub

    # Find bursts that overlap with SIB
    df_merged_intervals["SIB"] = 0
    for i, row in SIB[sub].iterrows():
        df_merged_intervals.loc[(df_merged_intervals["Start"] <= row["sib.end.time"]) & (df_merged_intervals["End"] >= row["sib.onset.time"]), "SIB"] = 1

    # Separate df_merged_intervals from start sleep to midpoint
    df_merged_intervals_1 = df_merged_intervals[df_merged_intervals["Start"] < sleep_midpoint]
    # Separate df_merged_intervals from midpoint to end sleep
    df_merged_intervals_2 = df_merged_intervals[df_merged_intervals["Start"] >= sleep_midpoint]

    limbs_comb_1 = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_1["Limbs"])
    limbs_comb_2 = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_2["Limbs"])

    limbs_combinations_df_1 = pd.DataFrame(limbs_comb_1.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_1["sub_ID"] = sub

    limbs_combinations_df_2 = pd.DataFrame(limbs_comb_2.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_2["sub_ID"] = sub

    limbs_combinations1part[sub] = limbs_combinations_df_1
    limbs_combinations2part[sub] = limbs_combinations_df_2

    # part 1 sleep and wake
    df_merged_intervals_1_sleep = df_merged_intervals_1[df_merged_intervals_1["SIB"] == 1]
    df_merged_intervals_1_awake = df_merged_intervals_1[df_merged_intervals_1["SIB"] == 0]

    limbs_comb_1_sleep = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_1_sleep["Limbs"])
    limbs_comb_1_awake = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_1_awake["Limbs"])

    limbs_combinations_df_1_sleep = pd.DataFrame(limbs_comb_1_sleep.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_1_sleep["sub_ID"] = sub
    limbs_combinations_df_1_awake = pd.DataFrame(limbs_comb_1_awake.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_1_awake["sub_ID"] = sub

    limbs_combinations1part_sleep[sub] = limbs_combinations_df_1_sleep
    limbs_combinations1part_awake[sub] = limbs_combinations_df_1_awake

    # part 2 sleep and wake
    df_merged_intervals_2_sleep = df_merged_intervals_2[df_merged_intervals_2["SIB"] == 1]
    df_merged_intervals_2_awake = df_merged_intervals_2[df_merged_intervals_2["SIB"] == 0]

    limbs_comb_2_sleep = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_2_sleep["Limbs"])
    limbs_comb_2_awake = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_2_awake["Limbs"])

    limbs_combinations_df_2_sleep = pd.DataFrame(limbs_comb_2_sleep.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_2_sleep["sub_ID"] = sub
    limbs_combinations_df_2_awake = pd.DataFrame(limbs_comb_2_awake.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_2_awake["sub_ID"] = sub

    limbs_combinations2part_sleep[sub] = limbs_combinations_df_2_sleep
    limbs_combinations2part_awake[sub] = limbs_combinations_df_2_awake


    #### DIVIDE SLEEP in three parts ####
    # sleep_upper_third = start_sleep + (end_sleep - start_sleep) / 3
    # sleep_lower_third = start_sleep + 2 * (end_sleep - start_sleep) / 3

    # df_merged_intervals_1 = df_merged_intervals[df_merged_intervals["Start"] < sleep_upper_third]
    # df_merged_intervals_2 = df_merged_intervals[(df_merged_intervals["Start"] >= sleep_upper_third) & (df_merged_intervals["Start"] < sleep_lower_third)]
    # df_merged_intervals_3 = df_merged_intervals[df_merged_intervals["Start"] >= sleep_lower_third]

    # limbs_comb_1 = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_1["Limbs"])
    # limbs_comb_2 = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_2["Limbs"])
    # limbs_comb_3 = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_3["Limbs"])

    # limbs_combinations_df_1 = pd.DataFrame(limbs_comb_1.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    # limbs_combinations_df_1["sub_ID"] = sub

    # limbs_combinations_df_2 = pd.DataFrame(limbs_comb_2.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    # limbs_combinations_df_2["sub_ID"] = sub

    # limbs_combinations_df_3 = pd.DataFrame(limbs_comb_3.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    # limbs_combinations_df_3["sub_ID"] = sub

    # limbs_combinations1part[sub] = limbs_combinations_df_1
    # limbs_combinations2part[sub] = limbs_combinations_df_2
    # limbs_combinations3part[sub] = limbs_combinations_df_3

limbs_combinations_df_ALL_1 = pd.concat(limbs_combinations1part.values(), ignore_index=True)
limbs_combinations_df_ALL_2 = pd.concat(limbs_combinations2part.values(), ignore_index=True)
limbs_combinations_df_sleep_ALL_1 = pd.concat(limbs_combinations1part_sleep.values(), ignore_index=True)
limbs_combinations_df_sleep_ALL_2 = pd.concat(limbs_combinations2part_sleep.values(), ignore_index=True)
limbs_combinations_df_awake_ALL_1 = pd.concat(limbs_combinations1part_awake.values(), ignore_index=True)
limbs_combinations_df_awake_ALL_2 = pd.concat(limbs_combinations2part_awake.values(), ignore_index=True)
# limbs_combinations_df_ALL_3 = pd.concat(limbs_combinations3part.values(), ignore_index=True)

# Combine the two DFs
limbs_combinations_df_ALL_1["part"] = "First Half"
limbs_combinations_df_ALL_2["part"] = "Second Half"
limbs_combinations_df_sleep_ALL_1["part"] = "First Half"
limbs_combinations_df_sleep_ALL_2["part"] = "Second Half"
limbs_combinations_df_awake_ALL_1["part"] = "First Half"
limbs_combinations_df_awake_ALL_2["part"] = "Second Half"

# limbs_combinations_df_ALL_3["part"] = "Third Half"
# limbs_combinations_df_1_and_2 = pd.concat([limbs_combinations_df_ALL_1, limbs_combinations_df_ALL_2], ignore_index=True)
# limbs_combinations_df_1_and_2_and_3 = pd.concat([limbs_combinations_df_ALL_1, limbs_combinations_df_ALL_2, limbs_combinations_df_ALL_3], ignore_index=True)

limbs_comb_groupby_1 = limbs_combinations_df_ALL_1.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_groupby_2 = limbs_combinations_df_ALL_2.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_groupby_sleep_1 = limbs_combinations_df_sleep_ALL_1.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_groupby_sleep_2 = limbs_combinations_df_sleep_ALL_2.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_groupby_awake_1 = limbs_combinations_df_awake_ALL_1.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_groupby_awake_2 = limbs_combinations_df_awake_ALL_2.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
# limbs_comb_groupby_3 = limbs_combinations_df_ALL_3.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_groupby = pd.concat((limbs_comb_groupby_1, limbs_comb_groupby_2))
limbs_comb_groupby_sleep = pd.concat((limbs_comb_groupby_sleep_1, limbs_comb_groupby_sleep_2))
limbs_comb_groupby_awake = pd.concat((limbs_comb_groupby_awake_1, limbs_comb_groupby_awake_2))
# limbs_comb_groupby = pd.concat((limbs_comb_groupby_1, limbs_comb_groupby_2, limbs_comb_groupby_3))

# Plot this information
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# sns.barplot(data = limbs_comb_groupby_1.reset_index(), x = "sub_ID", y = "Count", hue = "sub_ID")#, hue = "Limbs")
# plt.subplot(1, 2, 2, sharey = plt.subplot(1, 2, 1))
# sns.barplot(data = limbs_comb_groupby_2.reset_index(), x = "sub_ID", y = "Count", hue = "sub_ID")#, hue = "Limbs")

# plt.figure()
# plt.pie(limbs_combinations_df_ALL_1.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count'], labels = limbs_combinations_df_ALL_1.groupby('Limbs').sum().sort_values(by='Count', ascending=False).index, autopct='%1.1f%%');

# plt.figure()
# plt.pie(limbs_combinations_df_ALL_2.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count'], labels = limbs_combinations_df_ALL_2.groupby('Limbs').sum().sort_values(by='Count', ascending=False).index, autopct='%1.1f%%');

In [126]:
# Select only the 5 limbs movements

fiveL_sleep_1 = limbs_comb_groupby_sleep_1["Count"].unstack()[limbs_comb_groupby_sleep_1["Count"].unstack().index == limbs_comb_groupby_sleep_1["Count"].unstack().index[1]]
fiveL_sleep_2 = limbs_comb_groupby_sleep_2["Count"].unstack()[limbs_comb_groupby_sleep_2["Count"].unstack().index == limbs_comb_groupby_sleep_2["Count"].unstack().index[1]]

In [ ]:
fiveL_sleep_1.T

In [132]:
sleep_1 = [a.total_seconds() / 3600 for a in list(sleep_time_part1.values())]
sleep_2 = [a.total_seconds() / 3600 for a in list(sleep_time_part2.values())]

data1 = fiveL_sleep_1.values.squeeze() / sleep_1
data2 = fiveL_sleep_2.values.squeeze() / sleep_2

In [136]:
from scipy import stats

w_stat, p_value = stats.wilcoxon(data1, data2)

print(f"W-statistic: {w_stat}, p-value: {p_value}")

# boxplot with data1 and data2
plt.figure(figsize=(9, 5))
sns.boxplot(data = pd.DataFrame({"First Half": data1, "Second Half": data2}), palette = "Set2")

W-statistic: 1.0, p-value: 0.0078125


<Axes: >

In [84]:
limbs_comb_groupby_all_bursts_together = limbs_comb_groupby.groupby(["part", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_groupby_sleep_all_bursts_together = limbs_comb_groupby_sleep.groupby(["part", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_groupby_awake_all_bursts_together = limbs_comb_groupby_awake.groupby(["part", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)

plt.figure(figsize=(12, 6))
sns.barplot(data = limbs_comb_groupby_all_bursts_together.reset_index(), x = "sub_ID", y = "Count", hue = "part", errorbar=None, palette = "Set2")#, hue = "Limbs")
plt.legend(fancybox=True, frameon=True, shadow=True)
plt.ylabel("Number of bursts")
plt.title("Bursts distribution throughout the night")

Text(0.5, 1.0, 'Bursts distribution throughout the night')

**Scholle2014**: they observed a lack of clear structure of distribution of LMS per hour of sleep

**Ferri2008**: an important age effect can be observed with an evident decline of the number of PLMS per hour of sleep already present in subjects aged 15–25 years and persisting in the sub sequent decades, up to the age of 75 years; after this age,PLMS tend to be equally distributed across the entire night

In [85]:
limbs_comb_groupby_all_subjects_together = limbs_comb_groupby.groupby(["sub_ID", "part"]).sum()
limbs_comb_groupby_sleep_all_subjects_together = limbs_comb_groupby_sleep.groupby(["sub_ID", "part"]).sum()
limbs_comb_groupby_awake_all_subjects_together = limbs_comb_groupby_awake.groupby(["sub_ID", "part"]).sum()

plt.figure(figsize=(12, 6))
sns.boxplot(data = limbs_comb_groupby_all_subjects_together.reset_index(), x = "part", y = "Count", palette = "Set2")
plt.ylabel("Number of bursts")
plt.title("Bursts distribution throughout the night")

plt.figure(figsize=(12, 6))
sns.boxplot(data = limbs_comb_groupby_sleep_all_subjects_together.reset_index(), x = "part", y = "Count", palette = "Set2")
plt.ylabel("Number of bursts")
plt.title("Bursts distribution throughout the night - Sleep")

plt.figure(figsize=(12, 6))
sns.boxplot(data = limbs_comb_groupby_awake_all_subjects_together.reset_index(), x = "part", y = "Count", palette = "Set2")
plt.ylabel("Number of bursts")
plt.title("Bursts distribution throughout the night - Awake");

/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_24759/1195250631.py:6: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data = limbs_comb_groupby_all_subjects_together.reset_index(), x = "part", y = "Count", palette = "Set2")
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_24759/1195250631.py:11: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data = limbs_comb_groupby_sleep_all_subjects_together.reset_index(), x = "part", y = "Count", palette = "Set2")
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_24759/1195250631.py:16: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` an

Normalize to sleep duration

In [86]:
sns.set_context("talk")

In [87]:
limbs_comb_groupby_sleep_all_subjects_together = limbs_comb_groupby_sleep.groupby(["sub_ID", "part"]).sum()
data1 = []
data2 = []

# normalize to sleep duration

for sub in subjects:
    limbs_comb_groupby_sleep_all_subjects_together.loc[sub, "First Half"] /= (sleep_time_part1[sub].total_seconds() / 3600)
    data1.append(limbs_comb_groupby_sleep_all_subjects_together.loc[sub, "First Half"])
    limbs_comb_groupby_sleep_all_subjects_together.loc[sub, "Second Half"] /= (sleep_time_part2[sub].total_seconds() / 3600)
    data2.append(limbs_comb_groupby_sleep_all_subjects_together.loc[sub, "Second Half"])

plt.figure(figsize=(12, 6))
ax = sns.boxplot(data = limbs_comb_groupby_sleep_all_subjects_together.reset_index(), x = "part", y = "Count", palette = "Set2")
sns.stripplot(data = limbs_comb_groupby_sleep_all_subjects_together.reset_index(), x = "part", y = "Count", palette = "tab10", size = 12, alpha = 0.9,
              edgecolor = "black", linewidth = 1)

plt.xticks(fontsize = 32)
plt.ylabel("Number of bursts / sleep duration")
# Insert the asterisc for significant differences
x1, x2 = 0, 1   # columns 'Sat' and 'Sun' (first column: 0, see plt.xticks())
y, h, col = limbs_comb_groupby_sleep_all_subjects_together["Count"].max() + 1.5, 1.5, 'k'
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "p < 0.05", ha='center', va='bottom', color=col, fontsize = 19)
# plt.text((x1+x2)*.5, y+h, "**", ha='center', va='bottom', color=col, fontsize = 31)
plt.ylim(5, y+h+5)

/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_24759/1735717787.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '12.540983606557377' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  limbs_comb_groupby_sleep_all_subjects_together.loc[sub, "First Half"] /= (sleep_time_part1[sub].total_seconds() / 3600)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_24759/1735717787.py:14: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.boxplot(data = limbs_comb_groupby_sleep_all_subjects_together.reset_index(), x = "part", y = "Count", palette = "Set2")
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_24759/1735717787.py:15: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and wil

(5.0, 51.32567753789618)

In [78]:
plt.close("all")

Statistical test

In [89]:
[a[0] for a in data1]

/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_19210/4249030658.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  [a[0] for a in data1]


[12.540983606557377,
 18.89055472263868,
 16.641953772350632,
 9.72972972972973,
 18.682634730538922,
 10.408921933085502,
 12.17910447761194,
 22.166476624857467,
 22.23350253807107,
 27.110582639714625,
 10.77423552374756,
 21.504725897920604]

In [88]:
bursts1st_part = np.array([a[0] for a in data1])
bursts2nd_part = np.array([a[0] for a in data2])

/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_24759/3283389104.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  bursts1st_part = np.array([a[0] for a in data1])
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_24759/3283389104.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  bursts2nd_part = np.array([a[0] for a in data2])


In [89]:
from scipy import stats

w_stat, p_value = stats.wilcoxon(bursts1st_part, bursts2nd_part)

print(f"W-statistic: {w_stat}, p-value: {p_value}")

t_stat, p_value = stats.ttest_rel(bursts1st_part, bursts2nd_part)

print(f"t-statistic: {t_stat}, p-value: {p_value}")

W-statistic: 0.0, p-value: 0.00390625
t-statistic: -5.030077067230172, p-value: 0.001014067540332015


## Distribution of wake (GGIR) during the night

A wake episode counts as one

In [7]:
import pyreadr

In [8]:
diary_SPT = {    
    "158": [pd.Timestamp('2024-02-28 23:00:00'), pd.Timestamp('2024-02-29 07:15:00')], # 158 OK
    "633": [pd.Timestamp('2024-03-07 00:05:00'), pd.Timestamp('2024-03-07 06:36:00')], # 633 OK
    "906": [pd.Timestamp('2024-03-07 00:30:00'), pd.Timestamp('2024-03-07 07:30:00')], # 906 OK
    "958": [pd.Timestamp('2024-03-13 22:00:00'), pd.Timestamp('2024-03-14 06:00:00')], # 958 OK
    "127": [pd.Timestamp('2024-03-13 23:15:00'), pd.Timestamp('2024-03-14 06:50:00')], # 127 OK
    "098": [pd.Timestamp('2024-03-16 02:01:00'), pd.Timestamp('2024-03-16 09:50:00')], # 098 OK
    "547": [pd.Timestamp('2024-03-16 01:04:00'), pd.Timestamp('2024-03-16 07:40:00')], # 547 OK
    "815": [pd.Timestamp('2024-03-20 23:00:00'), pd.Timestamp('2024-03-21 07:30:00')], # 815 OK
    "914": [pd.Timestamp('2024-03-20 21:50:00'), pd.Timestamp('2024-03-21 05:50:00')], # 914 OK
    "971": [pd.Timestamp('2024-03-20 23:50:00'), pd.Timestamp('2024-03-21 07:50:00')], # 971 OK
    "279": [pd.Timestamp('2024-03-28 00:10:00'), pd.Timestamp('2024-03-28 07:27:00')], # 279 OK
    "965": [pd.Timestamp('2024-03-28 01:25:00'), pd.Timestamp('2024-03-28 09:20:00')], # 965 OK
}

diary_TIB = {
    "158": [pd.Timestamp('2024-02-28 22:15:00'), pd.Timestamp('2024-02-29 07:45:00')], # 158 OK
    "633": [pd.Timestamp('2024-03-06 23:39:00'), pd.Timestamp('2024-03-07 08:00:00')], # 633 OK 
    "906": [pd.Timestamp('2024-03-07 00:15:00'), pd.Timestamp('2024-03-07 07:35:00')], # 906 OK
    "958": [pd.Timestamp('2024-03-13 21:30:00'), pd.Timestamp('2024-03-14 06:30:00')], # 958 OK
    "127": [pd.Timestamp('2024-03-13 22:00:00'), pd.Timestamp('2024-03-14 07:10:00')], # 127 OK 
    "098": [pd.Timestamp('2024-03-16 01:49:00'), pd.Timestamp('2024-03-16 09:52:00')], # 098 OK 
    "547": [pd.Timestamp('2024-03-16 00:26:00'), pd.Timestamp('2024-03-16 08:20:00')], # 547 OK 
    "815": [pd.Timestamp('2024-03-20 22:00:00'), pd.Timestamp('2024-03-21 07:30:00')], # 815 OK 
    "914": [pd.Timestamp('2024-03-20 21:30:00'), pd.Timestamp('2024-03-21 06:20:00')], # 914 OK 
    "971": [pd.Timestamp('2024-03-20 23:30:00'), pd.Timestamp('2024-03-21 08:08:00')], # 971 OK 
    "279": [pd.Timestamp('2024-03-28 00:04:00'), pd.Timestamp('2024-03-28 07:41:00')], # 279 OK
    "965": [pd.Timestamp('2024-03-28 01:22:00'), pd.Timestamp('2024-03-28 09:22:00')], # 965 OK
}

In [10]:
part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms3.out/"
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815"]

SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

bursts_lw = {sub: 0 for sub in subjects}
bursts_rw = {sub: 0 for sub in subjects}
bursts_ll = {sub: 0 for sub in subjects}
bursts_rl = {sub: 0 for sub in subjects}
bursts_trunk = {sub: 0 for sub in subjects}
bursts_all_limbs = {sub: 0 for sub in subjects}
bursts_all_limbs_new = {sub: 0 for sub in subjects}
SIB = {sub: 0 for sub in subjects}

n1 = {sub: 0 for sub in subjects}
n2 = {sub: 0 for sub in subjects}
for i, sub in enumerate(subjects):
    SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
    # end time - onset time
    SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]
    # onset time - previous end time
    # print(min(SIB_all[sub].dropna()["awake.duration"]))
    # SIB[sub]['sib.onset.time'] += pd.Timedelta('5s')
    # SIB[sub]['sib.end.time'] += pd.Timedelta('5s')

    spt_start = diary_SPT[sub][0] - pd.Timedelta('10 min')
    spt_end = diary_SPT[sub][1] + pd.Timedelta('10 min')

    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    
    SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]

    SIB[sub]["sub_ID"] = sub

    sleep_midpoint = spt_start + (spt_end - spt_start) / 2
    # print(start_sleep, end_sleep, sleep_midpoint)

    SIB[sub]["part"] = np.where(SIB[sub]["sib.onset.time"] < sleep_midpoint, "First Half", "Second Half")

    number_awake_1stpart = len(SIB[sub][SIB[sub]["part"] == "First Half"])
    number_awake_2ndpart = len(SIB[sub][SIB[sub]["part"] == "Second Half"])

    n1[sub] = number_awake_1stpart
    n2[sub] = number_awake_2ndpart

awake_distr_df = pd.DataFrame({"sub_ID": list(n1.keys()), "First Half": list(n1.values()), "Second Half": list(n2.values())})

In [11]:
sns.set_context("talk")

plt.figure()
sns.boxplot(data = awake_distr_df, palette = "Set2")
plt.ylabel("Number of awake periods (GGIR)")
plt.title("Awake periods distribution during TIB throughout the night")

Text(0.5, 1.0, 'Awake periods distribution during TIB throughout the night')

In [15]:
from scipy import stats

w_stat, p_value = stats.wilcoxon(awake_distr_df["First Half"], awake_distr_df["Second Half"])

print(f"W-statistic: {w_stat}, p-value: {p_value}")

# t_stat, p_value = stats.ttest_rel(awake_distr_df["First Half"], awake_distr_df["Second Half"])

# print(f"t-statistic: {t_stat}, p-value: {p_value}")

W-statistic: 9.0, p-value: 0.2059032107320684


In [16]:
awake_distr_df

,sub_ID,First Half,Second Half
0,158,9,10
1,098,12,11
2,633,7,10
3,279,3,3
4,906,10,5
5,547,3,7
6,971,7,10
7,958,11,13
8,815,8,13


## New - Postures 

In [4]:
from collections import Counter

import pyreadr

from functions.bursts import characterize_bursts

In [7]:
part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms3.out/"
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815", "127", "914", "965"]

SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

bursts_lw = {sub: 0 for sub in subjects}
bursts_rw = {sub: 0 for sub in subjects}
bursts_ll = {sub: 0 for sub in subjects}
bursts_rl = {sub: 0 for sub in subjects}
bursts_trunk = {sub: 0 for sub in subjects}
bursts_all_limbs = {sub: 0 for sub in subjects}
bursts_all_limbs_new = {sub: 0 for sub in subjects}
SIB = {sub: 0 for sub in subjects}

bursts_df = pd.DataFrame()

for i, sub in enumerate(subjects):
    SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
    # end time - onset time
    SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]
    # onset time - previous end time
    # print(min(SIB_all[sub].dropna()["awake.duration"]))
    # SIB[sub]['sib.onset.time'] += pd.Timedelta('5s')
    # SIB[sub]['sib.end.time'] += pd.Timedelta('5s')
    if sub in ["127", "914", "965"]:
        with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_TIB.pkl', 'rb') as f:
            bursts = pickle.load(f)
    else:
        with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_TIB.pkl', 'rb') as f:
            bursts = pickle.load(f)

    df_merged_intervals = characterize_bursts(bursts)
    spt_start = diary_SPT[sub][0] - pd.Timedelta('10 min')
    spt_end = diary_TIB[sub][1] + pd.Timedelta('5 min')

    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)

    # Take df_merged_intervals between spt_start and spt_end
    df_merged_intervals = df_merged_intervals[(df_merged_intervals["Start"] >= spt_start) & (df_merged_intervals["End"] <= spt_end)].reset_index(drop=True) 

    SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]
    SIB[sub]["sub_ID"] = sub

    df_merged_intervals["SIB"] = 0
    for i, row in SIB[sub].iterrows():
        df_merged_intervals.loc[(df_merged_intervals["Start"] >= row["sib.onset.time"] + pd.Timedelta("5s")) & (df_merged_intervals["End"] <= row["sib.end.time"] - pd.Timedelta("5s")), "SIB"] = 1

    df_merged_intervals["sub_ID"] = sub

    bursts_df = pd.concat([bursts_df, df_merged_intervals])

bursts_df = bursts_df.reset_index(drop=True)

In [63]:
bursts_LW = bursts_df[(bursts_df["Limbs"] == {"LW"})]["sub_ID"].value_counts()
bursts_RW = bursts_df[(bursts_df["Limbs"] == {"RW"})]["sub_ID"].value_counts()
bursts_LL = bursts_df[(bursts_df["Limbs"] == {"LL"})]["sub_ID"].value_counts()
bursts_RL = bursts_df[(bursts_df["Limbs"] == {"RL"})]["sub_ID"].value_counts()

bursts_single_limbs =  bursts_df[(bursts_df["Limbs"] == {"LW"}) | (bursts_df["Limbs"] == {"RW"}) | (bursts_df["Limbs"] == {"RL"}) | (bursts_df["Limbs"] == {"LL"})].reset_index(drop=True)

bursts_single_limbs["sub_ID"].value_counts().mean(), bursts_LW.mean() + bursts_RW.mean() + bursts_LL.mean() + bursts_RL.mean()

(72.41666666666667, 73.68181818181819)

In [60]:
bursts_df[(bursts_df["Limbs"] == {'LL', 'LW', 'RL', 'RW', 'T'})]["sub_ID"].value_counts().mean()

# Find single limb movements

96.33333333333333

In [52]:
bursts_df[bursts_df["PC"] > 5]["Limbs"].value_counts()

Limbs
{LW, T, LL, RL, RW}    301
Name: count, dtype: int64

In [44]:
# all 5 limbs

bursts_df_allLimbs = bursts_df[bursts_df["Limbs"] != {'LL', 'LW', 'RL', 'RW', 'T'}].reset_index(drop=True)

bursts_df_allLimbs[bursts_df_allLimbs["PC"] > 30].shape[0] / bursts_df_allLimbs.shape[0]

0.0

In [36]:
# Find the number of posture changes > 30 degrees for each subject

PCs = {sub: 0 for sub in subjects}
for sub in subjects:
    PCs[sub] = bursts_df_allLimbs[bursts_df_allLimbs["sub_ID"] != sub][bursts_df_allLimbs["PC"] > 10].shape[0] / bursts_df_allLimbs[bursts_df_allLimbs["sub_ID"] == sub].shape[0]

/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_19210/71862254.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  PCs[sub] = bursts_df_allLimbs[bursts_df_allLimbs["sub_ID"] != sub][bursts_df_allLimbs["PC"] > 10].shape[0] / bursts_df_allLimbs[bursts_df_allLimbs["sub_ID"] == sub].shape[0]
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_19210/71862254.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  PCs[sub] = bursts_df_allLimbs[bursts_df_allLimbs["sub_ID"] != sub][bursts_df_allLimbs["PC"] > 10].shape[0] / bursts_df_allLimbs[bursts_df_allLimbs["sub_ID"] == sub].shape[0]
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_19210/71862254.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  PCs[sub] = bursts_df_allLimbs[bursts_df_allLimbs["sub_ID"] != sub][bursts_df_allLimbs["PC"] > 10].shape[0] / bursts_df_allLimbs[bursts_df_allLimbs["sub_ID"] == sub].shape[0]
/var

In [38]:
# mean

PCs

{'158': 0.0,
 '098': 0.0,
 '633': 0.0,
 '279': 0.0,
 '906': 0.0,
 '547': 0.0,
 '971': 0.0,
 '958': 0.0,
 '815': 0.0,
 '127': 0.0,
 '914': 0.0,
 '965': 0.0}

## Barcode

In [8]:
bursts_df_GPT = bursts_df[bursts_df["sub_ID"] == "815"]
bursts_df_GPT.drop("sub_ID", axis = 1, inplace = True)

/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_39793/2749981866.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bursts_df_GPT.drop("sub_ID", axis = 1, inplace = True)


In [16]:
bursts_df_GPT.to_csv("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/bursts_df_GPT.csv", index = False)

In [121]:
bursts_df_GPT.reset_index(drop=True, inplace = True)

In [118]:
import matplotlib.pyplot as plt

from matplotlib._enums import CapStyle

CapStyle.demo()
plt.show()

In [9]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as mpatches

# Determine the number of limbs moving in each entry
bursts_df_GPT['Num_Limbs'] = bursts_df_GPT['Limbs'].apply(lambda x: len(x))

# Create a color mapping for different numbers of limbs
colors = {
    1: 'blue',
    2: 'green',
    3: sns.color_palette()[5],
    4: 'aquamarine',
    5: 'red'
}

# Generate a plot with a color barcode representing the activity over time
fig, ax = plt.subplots(figsize=(15, 6))

for idx, row in bursts_df_GPT.iterrows():
    start = row['Start']
    end = row['End']
    num_limbs = row['Num_Limbs']
    color = colors.get(num_limbs, 'black')  # Default to black if an unexpected number of limbs is found
    ax.plot([start, end], [1, 1], color=color, linewidth=126, solid_capstyle='butt')

SIB_965 = SIB["815"]
# ax.plot([diary_SPT["965"][0], SIB_965["sib.onset.time"].iloc[0]], [1.0005, 1.0005], color="black", linewidth=12, solid_capstyle='butt')
plt.axvspan(diary_SPT["815"][0], SIB_965["sib.onset.time"].iloc[0], color = "black", alpha = 0.2)
for i in range(len(SIB_965)-1):
    # plt.axvspan(SIB_965["sib.end.time"].iloc[i], SIB_965["sib.onset.time"].iloc[i+1], color = "green", alpha = 0.5)
    start_awake = SIB_965["sib.end.time"].iloc[i]
    end_awake = SIB_965["sib.onset.time"].iloc[i+1]
    # ax.plot([start_awake, end_awake], [1.0005, 1.0005], color="black", linewidth=12, solid_capstyle='butt')
    plt.axvspan(SIB_965["sib.end.time"].iloc[i], SIB_965["sib.onset.time"].iloc[i+1], color = "black", alpha = 0.2)

plt.axvline(x = diary_SPT["815"][0], color = "black", linestyle = "--", linewidth = 1.6)
plt.axvline(x = diary_SPT["815"][1], color = "black", linestyle = "--", linewidth = 1.6)

legend_patches = [mpatches.Patch(color=color, label=f'{num}') for num, color in colors.items()]
legend_patches.append(mpatches.Patch(color='black', alpha = 0.2, label='Awake Periods'))
plt.legend(handles=legend_patches, loc='upper right', ncol = 6, fontsize = 19, fancybox=True, frameon=True, shadow=True)

# Format the x-axis for better readability
ax.xaxis.set_major_locator(mdates.HourLocator())
ax.xaxis.set_minor_locator(mdates.MinuteLocator(interval=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

# Set plot labels and title
# ax.set_xlabel('Time')
ax.set_ylim(0.9992, 1.0008)
plt.xticks(fontsize = 19, rotation=45)
ax.set_yticks([])
ax.set_title('Limb Movements During SPT (Color Coded by Number of Limbs)')

/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_39793/2374616834.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bursts_df_GPT['Num_Limbs'] = bursts_df_GPT['Limbs'].apply(lambda x: len(x))


Text(0.5, 1.0, 'Limb Movements During SPT (Color Coded by Number of Limbs)')

In [133]:
plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/barcode.png", dpi = 300, bbox_inches = 'tight')

In [98]:
# LEGEND

import matplotlib.patches as mpatches

# Create a legend with a color patch for each number of limbs

legend_patches = [mpatches.Patch(color=color, label=f'{num} Limbs') for num, color in colors.items()]
plt.legend(handles=legend_patches, loc='upper right', fontsize = 19)

plt.show()

In [97]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Create a new column in the dataframe that contains movement intensity, based on the percentiles of the AUC
bursts_df_GPT['Movement Intensity'] = pd.qcut(bursts_df_GPT['AUC'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])

# Create a color mapping for different numbers of limbs
colors = {
    'Low': 'blue',
    'Medium': 'green',
    'High': 'aquamarine',
    'Very High': 'red'
}

# Generate a plot with a color barcode representing the activity over time
fig, ax = plt.subplots(figsize=(15, 6))

for idx, row in bursts_df_GPT.iterrows():
    start = row['Start']
    end = row['End']
    movement_intensity = row['Movement Intensity']
    color = colors.get(movement_intensity, 'black')  # Default to black if an unexpected number of limbs is found
    ax.plot([start, end], [1, 1], color=color, linewidth=126, solid_capstyle='butt')

SIB_965 = SIB["965"]
# ax.plot([diary_SPT["965"][0], SIB_965["sib.onset.time"].iloc[0]], [1.0005, 1.0005], color="black", linewidth=12, solid_capstyle='butt')
plt.axvspan(diary_SPT["965"][0], SIB_965["sib.onset.time"].iloc[0], color = "black", alpha = 0.2)
for i in range(len(SIB_965)-1):
    # plt.axvspan(SIB_965["sib.end.time"].iloc[i], SIB_965["sib.onset.time"].iloc[i+1], color = "green", alpha = 0.5)
    start_awake = SIB_965["sib.end.time"].iloc[i]
    end_awake = SIB_965["sib.onset.time"].iloc[i+1]
    # ax.plot([start_awake, end_awake], [1.0005, 1.0005], color="black", linewidth=12, solid_capstyle='butt')
    plt.axvspan(SIB_965["sib.end.time"].iloc[i], SIB_965["sib.onset.time"].iloc[i+1], color = "black", alpha = 0.2)

# Format the x-axis for better readability
ax.xaxis.set_major_locator(mdates.HourLocator())
ax.xaxis.set_minor_locator(mdates.MinuteLocator(interval=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

plt.axvline(x = diary_SPT["965"][0], color = "black", linestyle = "--", linewidth = 1.1)
plt.axvline(x = diary_SPT["965"][1], color = "black", linestyle = "--", linewidth = 1.1)

# Set plot labels and title
ax.set_xlabel('Time')
ax.set_ylim(0.999, 1.0010)
ax.set_xlim(diary_SPT["965"][0] - pd.Timedelta(minutes=10), diary_SPT["965"][1] + pd.Timedelta(minutes=10))
plt.xticks(fontsize = 19, rotation=45)
ax.set_yticks([])
# ax.set_title('Limb Movements Over Time (Color Coded by Movement Intensity)')

/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_19210/1747179373.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bursts_df_GPT['Movement Intensity'] = pd.qcut(bursts_df_GPT['AUC'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


[]

In [80]:
SIB_965 = SIB["965"]

plt.figure()
for i, row in SIB_965.iterrows():
    plt.axvspan(row["sib.onset.time"], row["sib.end.time"], color = "red", alpha = 0.5)

# plot  awake
# plt.figure()
for i in range(len(SIB_965)-1):
    plt.axvspan(SIB_965["sib.end.time"].iloc[i], SIB_965["sib.onset.time"].iloc[i+1], color = "green", alpha = 0.5)

In [78]:
range(len(SIB_965) - 1)

range(0, 13)

In [81]:
SIB_965

,sib.onset.time,sib.end.time,sib.duration,awake.duration,sub_ID
0,2024-03-28 01:41:45,2024-03-28 02:42:20,0 days 01:00:35,0 days 00:17:35,965
1,2024-03-28 02:59:55,2024-03-28 03:05:30,0 days 00:05:35,0 days 00:00:15,965
2,2024-03-28 03:05:45,2024-03-28 03:23:15,0 days 00:17:30,0 days 00:00:15,965
3,2024-03-28 03:23:30,2024-03-28 05:16:50,0 days 01:53:20,0 days 00:01:05,965
4,2024-03-28 05:17:55,2024-03-28 05:41:20,0 days 00:23:25,0 days 00:03:35,965
5,2024-03-28 05:44:55,2024-03-28 06:24:55,0 days 00:40:00,0 days 00:04:00,965
6,2024-03-28 06:28:55,2024-03-28 06:44:15,0 days 00:15:20,0 days 00:02:25,965
7,2024-03-28 06:46:40,2024-03-28 07:04:55,0 days 00:18:15,0 days 00:00:15,965
8,2024-03-28 07:05:10,2024-03-28 07:55:50,0 days 00:50:40,0 days 00:02:30,965
9,2024-03-28 07:58:20,2024-03-28 08:05:10,0 days 00:06:50,0 days 00:04:20,965
